In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SVMSMOTE

from pathlib import Path

from ftplib import FTP

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-dark-palette')
%matplotlib inline

Below is code to automatically grab a file from the FTP for RNAcentral. FTPlib doesn't like large files though, so I ended up doing it manually. 

In [ ]:
# HOSTNAME = 'ftp.ebi.ac.uk'

# ftp = FTP(HOSTNAME)

# ftp.login()

# ftp.cwd('pub/databases/RNAcentral/current_release/json')

# filename = 'ensembl-xref-1-100001.json'

# with open(filename, "w") as rna_json:
#     ftp.retrlines(f"RETR {filename}", rna_json.write)


Instead of automated requests, I grabbed a few of the JSON files from the FTP, which can be read in with the following function. 

In [2]:
def load_rna(fname, inc_other=True, merge_other=True, simple_descrips=False, drop=True):
    rna = pd.read_json(fname)
    
    # drop extra columns which reference external databases -- always
    rna = rna.drop(['xrefs', 'md5'], axis=1)
    
    # make the column names pretty
    cols = {'rnacentral_id': 'RNA ID', 'description': 'Description', 'sequence':'Sequence', 'rna_type': 'RNA Type', 'taxon_id': 'Taxon ID'}
    rna = rna.rename(columns=cols)
    
    # remove underscores from RNA type
    rna['RNA Type'] = rna.apply(lambda row : (row['RNA Type'].replace('_', ' ')), axis=1)
    
    # combine the 'other', 'misc', 'nc', and 'lnc' types\n",
    if merge_other:
        rna['RNA Type'] = rna.apply(lambda row : (row['RNA Type'].replace('misc RNA', 'other')), axis=1)
        rna['RNA Type'] = rna.apply(lambda row : (row['RNA Type'].replace('lncRNA', 'other')), axis=1)
        rna['RNA Type'] = rna.apply(lambda row : (row['RNA Type'].replace('ncRNA', 'other')), axis=1)

    # drop rows with unclassified RNAs? 
    if not inc_other:
        rna = rna[rna['RNA Type'] != 'other']
        
        if not merge_other:
            rna = rna[rna['RNA Type'] != 'misc RNA']
            rna = rna[rna['RNA Type'] != 'ncRNA']
            rna = rna[rna['RNA Type'] != 'lncRNA']
    
    # Remove everything except latin name from species description
    if simple_descrips:
        rna['Description'] = rna.apply(lambda row : (row['Description'].split(' ')[0] + ' ' + row['Description'].split(' ')[1]), axis=1)\
        
    # drop extra columns with unnecessary info -- optional
    if drop:
        rna = rna.drop(['Taxon ID', 'RNA ID', 'Description'], axis=1)
    
    return rna



In [3]:
# load a batch of data files from RNAcentral or load in a single file?
load_all = True

if load_all:
    p = Path()
    p = p / 'data'
    files = p.glob('ensembl-*')
    rna = pd.DataFrame()

    for file in files:
        try: 
            print(f'Loading data from {str(file)}')
            rna = pd.concat([rna, load_rna(str(file), inc_other=False)])
        except:
            print(f'Could not load data from {str(file)}')

    print('All data loaded.')

else:
    rna = load_rna('data\ensembl-xref-34100001-34200001.json')


Loading data from data\ensembl-xref-10000001-10100001.json
Loading data from data\ensembl-xref-16500001-16600001.json
Loading data from data\ensembl-xref-22600001-22700001.json
Loading data from data\ensembl-xref-29100001-29200001.json
Loading data from data\ensembl-xref-29200001-29300001.json
Loading data from data\ensembl-xref-34100001-34200001.json
Loading data from data\ensembl-xref-6800001-6900001.json
Loading data from data\ensembl-xref-6900001-7000001.json
Loading data from data\ensembl-xref-7000001-7100001.json
Loading data from data\ensembl-xref-7100001-7200001.json
Loading data from data\ensembl-xref-7200001-7300001.json
Loading data from data\ensembl-xref-7300001-7400001.json
Loading data from data\ensembl-xref-7400001-7500001.json
Loading data from data\ensembl-xref-9400001-9500001.json
All data loaded.


In [4]:
def contract_families(types):
    
    classification = []
    
    classes = {'cis-reg': ['IRES', 'leader', 'riboswitch'], 
               'intron': ['autocatalytically spliced intron', 'intron gpI', 'intron gpII', 'intron'],
                  'gene': ['tRNA', 'rRNA', 'snRNA', 'snoRNA', 'pre miRNA', 'hammerhead ribozyme', 'SRP RNA', 'sRNA', 'Y RNA',
                          'RNase P RNA', 'lncRNA', 'tmRNA', 'RNase MRP RNA', 'vault RNA', 'ribozyme', 'miRNA', 'antisense RNA',
                          'telomerase RNA', 'scRNA', 'precursor RNA', 'scaRNA', 'piRNA'], 'artificial': ['guide RNA', 'siRNA']}

    for t in types:
        flag = False
        for c  in classes.keys():
            if t in classes[c]:
                classification.append(c)
                flag = True
                break

        if not flag:
            classification.append(t)


    family = []
    
    families = {'tRNA': ['tRNA', 'tmRNA'], 'rRNA': ['rRNA', 'SRP RNA', 'Y RNA'], 'snRNA': 'snRNA', 
                'snoRNA': ['snoRNA', 'scRNA', 'scaRNA'], 'miRNA': ['pre miRNA', 'miRNA', 'piRNA'], 'precursor RNA': ['precursor RNA'],
                'ribozyme': ['hammerhead ribozyme', 'RNase P RNA', 'RNase MRP RNA', 'ribozyme', 'autocatalytically spliced intron'], 
                'sRNA': ['sRNA'], 'antisense': ['antisense'], 'vault RNA': ['vault RNA'], 'telomerase RNA': ['telomerase RNA'],
                'artificial': ['guide RNA', 'siRNA']}

    for t in types:
        flag = False
        for f in families.keys():
            if t in families[f]:
                family.append(f)
                flag = True
                break

        if not flag:
            family.append(t)
            
    return classification, family

In [12]:
rna['RNA Class'], rna['RNA Family'] = contract_families(rna['RNA Type'])
rna.head()

,Sequence,RNA Type,RNA Class,RNA Family
0,GGAGGGTAAGCTCAGTGGTTAGAATACATGCTTAGCATGCAGGAGG...,tRNA,gene,tRNA
3,ATGCTAGCTTCAGCAGCACATATGCTAAAATTGGAATGATACAGAG...,snRNA,gene,snRNA
4,GCCGGGTGTGGTGGCTCACGCCTGTAATCCCAATAGTTTGGGAGGC...,sRNA,gene,sRNA
5,GCCGGGTGTGGTGGCTCACGCCTGTAATCCCAATAGTTTGGGAGGC...,sRNA,gene,sRNA
6,GCCGGGTGTGGCGGCGTGCACCTGTAGTCCCAGTTACTTGGAGGCT...,SRP RNA,gene,rRNA


In [14]:
merge_sequence = rna.groupby("Sequence").agg({'RNA Family':'first'}).reset_index()
print(len(rna))
print(len(merge_sequence))
merge_sequence

734593
595938


,Sequence,RNA Family
0,AAAAAAAAAAAAAAAAAAAAAAAATCCCTTTTTTTTTTTTTTTTTT...,ribozyme
1,AAAAAAAAAAAAAAAAAAAAAGAAATGCCCGATCTCATCTGAATTC...,rRNA
2,AAAAAAAAAAAAAAAAAAAAATACTGGGATCAGGCAGCCAGTGGGT...,snRNA
3,AAAAAAAAAAAAAAAAAAAAATACTGGGATCAGGCAGCCAGTGGGT...,snRNA
4,AAAAAAAAAAAAAAAGGACCATACTGGGATCAGCCAGCCAGTGGGT...,snRNA
...,...,...
595933,YATTAAACGCTGGCGGCAGCCTTAACACATGCAAGTCGAGCGGATG...,rRNA
595934,YCTTTGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTG...,rRNA
595935,YGGCAATGATGTTAYGAATTTCTTCACCTGAGCTCAAAGTGAAGAG...,snoRNA
595936,YGTTCAGCTTCTAATCGTCTCAGGACAATTTTTTGAACGC,rRNA


Feature engineering below includes: sequence length, nucleic acid ratios (ratios of A:T:C:G), ...

In [15]:
merge_sequence['Sequence len'] = merge_sequence['Sequence'].str.len()

def nucleic_acid_ratios(target):
    A = 0
    C = 0
    T = 0
    G = 0
    N = 0
    bad_vals = 0;
    for ch in target:
        if (ch == "A"):
            A += 1
        elif (ch == 'C'):
            C += 1;
        elif (ch == 'T'):
            T += 1;
        elif (ch == 'G'):
            G += 1;
        elif (ch == 'N'):
            N += 1;
        else:
            bad_vals += 1

    seq_len = len(target)
    a_rat = A / seq_len
    c_rat = C / seq_len
    t_rat = T / seq_len
    g_rat = G / seq_len
    n_rat = N / seq_len
    
    return a_rat, c_rat, t_rat, g_rat, n_rat;



In [16]:
merge_sequence["A Ratio"] = 0
merge_sequence["C Ratio"] = 0
merge_sequence["T Ratio"] = 0
merge_sequence["G Ratio"] = 0
merge_sequence["N Ratio"] = 0


for i in range (len(seq)):
    a_rat, c_rat, t_rat, g_rat, n_rat = nucleic_acid_ratios(seq[i])
    merge_sequence.loc[i, "A Ratio"] = a_rat
    merge_sequence.loc[i, "C Ratio"] = c_rat
    merge_sequence.loc[i, "T Ratio"] = t_rat
    merge_sequence.loc[i, "G Ratio"] = g_rat
    merge_sequence.loc[i, "N Ratio"] = n_rat

merge_sequence.head()

,Sequence,RNA Family,Sequence len,A Ratio,C Ratio,T Ratio,G Ratio,N Ratio
0,AAAAAAAAAAAAAAAAAAAAAAAATCCCTTTTTTTTTTTTTTTTTT...,ribozyme,168,0.363095,0.136905,0.404762,0.095238,0.0
1,AAAAAAAAAAAAAAAAAAAAAGAAATGCCCGATCTCATCTGAATTC...,rRNA,119,0.369748,0.159664,0.201681,0.268908,0.0
2,AAAAAAAAAAAAAAAAAAAAATACTGGGATCAGGCAGCCAGTGGGT...,snRNA,163,0.282209,0.226994,0.220859,0.269939,0.0
3,AAAAAAAAAAAAAAAAAAAAATACTGGGATCAGGCAGCCAGTGGGT...,snRNA,163,0.282209,0.226994,0.226994,0.263804,0.0
4,AAAAAAAAAAAAAAAGGACCATACTGGGATCAGCCAGCCAGTGGGT...,snRNA,163,0.257669,0.239264,0.226994,0.276074,0.0


In [17]:
x = merge_sequence.loc[:, ['Sequence len', 'A Ratio', 'C Ratio', 'T Ratio', 'G Ratio', 'N Ratio']]
y = merge_sequence.loc[:, 'RNA Family']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

x_train.head()

,Sequence len,A Ratio,C Ratio,T Ratio,G Ratio,N Ratio
342158,70,0.242857,0.214286,0.271429,0.271429,0.0
170446,73,0.219178,0.273973,0.191781,0.315068,0.0
16113,433,0.272517,0.230947,0.207852,0.288684,0.0
221684,489,0.269939,0.210634,0.222904,0.296524,0.0
161431,69,0.217391,0.217391,0.246377,0.318841,0.0


In [18]:
x_resample, y_resample = SVMSMOTE(k_neighbors=2).fit_resample(x_train, y_train)

KeyboardInterrupt: 

In [ ]:
rtypcount = y_resample.value_counts()
mtypcount = y_train.value_counts()
typcount = rna['RNA Family'].value_counts()
labs = y_resample.value_counts().index
xpos = np.arange(0, 3*len(labs), 3)

plt.figure(figsize=[12.5, 12.5])
plt.bar(x=xpos-0.8, height=typcount, log=True, width=0.8, label='Original')
plt.bar(x=xpos, height=mtypcount, log=True, width=0.8, tick_label=labs, label='Merged')
plt.bar(x=xpos+0.8, height=rtypcount, log=True, width=0.8, label='Resampled')

plt.legend()
plt.tight_layout()
plt.xticks(rotation=90)
plt.ylabel('Log count');

In [ ]:
ridge = RidgeClassifier().fit(x_train, y_train)
ridge.score(x_train, y_train)

In [ ]:
y_pred = ridge.predict(x_test)
fig, ax = plt.subplots(figsize=(12.5, 12.5))
metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred, normalize='pred', ax=ax, xticks_rotation='vertical', cmap='BuGn')
plt.title('Multinomial Ridge Regression Confusion Matrix')
plt.show()

In [ ]:
rfc = RandomForestClassifier().fit(x_train, y_train)
rfc.score(x_train, y_train)

In [ ]:
y_pred_rfc = rfc.predict(x_test)
fig, ax = plt.subplots(figsize=(12.5, 12.5))
metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rfc, normalize='pred', ax=ax, xticks_rotation='vertical', cmap='BuGn')
plt.title('Random Forest Classifier Confusion Matrix')
plt.show()

In [ ]:
def multiclass_metrics(y_test, y_pred):
    prec = metrics.precision_score(y_test, y_pred, average='weighted', zero_division=0)
    rec = metrics.recall_score(y_test, y_pred, average='weighted', zero_division=0)
    acc = metrics.balanced_accuracy_score(y_test, y_pred)
    
    print(f'Precision score: {prec}')
    print(f'Recall score: {rec}')
    print(f'Balanced accuracy score: {acc}')

In [ ]:
print('Ridge classifier metrics:')
multiclass_metrics(y_test, y_pred)
print('\nRandom Forest classifier metrics:')
multiclass_metrics(y_test, y_pred_rfc)

In [ ]:
rfc_resampled = RandomForestClassifier().fit(x_resample, y_resample)
rfc.score(x_resample, y_resample)

In [ ]:
y_pred_rfcr = rfc_resampled.predict(x_test)
fig, ax = plt.subplots(figsize=(12.5, 12.5))
metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rfcr, normalize='pred', ax=ax, xticks_rotation='vertical', cmap='BuGn')
plt.title('Random Forest Classifier Confusion Matrix with Resampling')
plt.show()

In [ ]:
multiclass_metrics(y_pred, y_pred_rfcr)